# Importing the libraries

In [97]:
import numpy as np
import pandas as pd
import tensorflow as tf

# Data Preprocessing

### Import the dataset

In [98]:
dataset = pd.read_csv('data/Churn_Modelling.csv')

In [99]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [100]:
X = dataset.iloc[:, 3:-1].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [101]:
y = dataset.iloc[:, -1].values
y

array([1, 0, 1, ..., 1, 1, 0])

### Encoding categorical data

In [102]:
# Encode gender
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [103]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [104]:
# Encode Geography
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Training and Test Sets

In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

In [106]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Build the ANN

In [107]:
# Init
ann = tf.keras.models.Sequential()

In [108]:
# Input layer and first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [109]:
# Second hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [110]:
# Output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Train the ANN

In [111]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [112]:
ann.fit(X_train, y_train, batch_size=32, epochs=50)

Epoch 1/50
250/250 [==============================] - 0s 322us/step - loss: 0.6618 - accuracy: 0.6301
Epoch 2/50
250/250 [==============================] - 0s 302us/step - loss: 0.5118 - accuracy: 0.7958
Epoch 3/50
250/250 [==============================] - 0s 296us/step - loss: 0.4547 - accuracy: 0.7961
Epoch 4/50
250/250 [==============================] - 0s 296us/step - loss: 0.4262 - accuracy: 0.8174
Epoch 5/50
250/250 [==============================] - 0s 299us/step - loss: 0.4144 - accuracy: 0.8242
Epoch 6/50
250/250 [==============================] - 0s 296us/step - loss: 0.4075 - accuracy: 0.8257
Epoch 7/50
250/250 [==============================] - 0s 295us/step - loss: 0.4023 - accuracy: 0.8276
Epoch 8/50
250/250 [==============================] - 0s 295us/step - loss: 0.3979 - accuracy: 0.8298
Epoch 9/50
250/250 [==============================] - 0s 293us/step - loss: 0.3942 - accuracy: 0.8317
Epoch 10/50
250/250 [==============================] - 0s 292us/step - loss: 0.391

# Make predictions and evaluate model

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [120]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 10ms/step
[[False]]


### Predicting the Test set results

In [121]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 267us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]
